In [1]:
# Fitted Q iteration (Expected SARSA)

In [2]:
# policy_train_path = '../continuous/dqn_normal/dqn_normal_actions_train.p'
# policy_val_path = '../continuous/dqn_normal/dqn_normal_actions_val.p'
# policy_test_path = '../continuous/dqn_normal/dqn_normal_actions_test.p'

In [3]:
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import cPickle as pickle
import copy

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
with open('../data/state_features.txt') as f:
    state_features = f.read().split()
print (state_features)
print len(state_features)

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission', 'bloc']
48


In [6]:
df = pd.read_csv('../data/rl_train_data_final_cont.csv')

In [7]:
df.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,mortality_90d,Weight_kg,...,median_dose_vaso,max_dose_vaso,input_total_tev,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev,vaso_input,iv_input,reward
0,0.000000,3,7245052800,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.797351,0.939195,0.589916,0.750908,0.554500,0.0,4.0,0.125000
1,0.222560,3,7245067200,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.831780,0.934543,0.674384,0.819589,0.580033,0.0,4.0,0.657321
2,0.356608,3,7245081600,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.833222,0.656575,0.765423,0.939329,0.555033,0.0,2.0,1.367788
3,0.452837,3,7245096000,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.834033,0.603831,0.783597,0.847073,0.545700,0.0,2.0,1.199099
4,0.527957,3,7245110400,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.834836,0.603831,0.794059,0.811583,0.539533,0.0,2.0,1.057596


In [8]:
val_df = pd.read_csv('../data/rl_val_data_final_cont.csv')

In [9]:
test_df = pd.read_csv('../data/rl_test_data_final_cont.csv')

In [10]:
# # Here we load the actions for the policy we want to evaluate into the relevant dataframes
# policy_train = pickle.load(open(policy_train_path, "rb" ))
# policy_test = pickle.load(open(policy_test_path, "rb" ))
# policy_val = pickle.load(open(policy_val_path, "rb" ))

# df['policy'] = np.array(policy_train)
# test_df['policy'] = np.array(policy_test)
# val_df['policy'] = np.array(policy_val)


save_dir = './MixNN_EM/'
train_actions = np.load(save_dir + "cond_lik_train_dr.npy")
val_actions = np.load(save_dir + "cond_lik_val_dr.npy")
test_actions = np.load(save_dir + "cond_lik_test_dr.npy")

mix_comp_eval = 3 # SET THIS ACCORDINGLY
train_actions = [np.array(list(i)) for i in train_actions[:,:,mix_comp_eval]]
val_actions = [np.array(list(i)) for i in val_actions[:,:,mix_comp_eval]]
test_actions = [np.array(list(i)) for i in test_actions[:,:,mix_comp_eval]]

df['policy'] = train_actions
val_df['policy'] = val_actions
test_df['policy'] = test_actions

In [11]:
fq_state_terms = copy.deepcopy(state_features)
fq_state_terms.append('iv_input')
fq_state_terms.append('vaso_input')

In [12]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [13]:
action_tuple = []
for iv in range(5):
    for vaso in range(5):
        action_tuple.append([iv,vaso])
action_tuple = np.array(action_tuple)

In [14]:
# first define all the (s,a) pairs
# then create (s', a') for each possible next action. This is of size (num_points*25)
# call predict on each of these 
# reshape into (-1,25): (num_points, 25)
# multiply elementwise by prob_actions
# reduce sum along 1 axis to get expected Q values

In [15]:
def make_s_a(df_in):
    cur_s_a = df_in[fq_state_terms].values
    return cur_s_a

In [16]:
# save which indices of the df are terminal
def find_term_indices(df_in):
    term = []
    df_local = df_in.reset_index()
    for count, i in enumerate(df_local.index):
        if count % 10000 == 0 and count > 0:
            print "find_term_indices, step %d" % count
        if i != df_local.index[-1]:
            if df_local.loc[i, 'icustayid'] != df_local.loc[i+1, 'icustayid']:
                term.append(count)
        else:
            # last state is the terminal one
            term.append(count)
    return np.array(term)       

In [17]:
# construct s', a' for all a in action space
def make_s_next_a_next(df_in):
    df_local = df_in.reset_index()
    s_next = df_local[fq_state_terms].values
    initial_shape = s_next.shape
    
    # add padding at the end
    s_next = np.append(s_next, [np.zeros(initial_shape[1])], axis=0)
    # remove first elem
    s_next = s_next[1:]  
    
    # tile in 0 dimension to get the different actions
    s_next_all_actions = np.repeat(s_next, 25, axis=0)
    
    
    act_tup_tiled = np.tile(action_tuple, [initial_shape[0],1])
    
    s_next_all_actions[:,-2:] = act_tup_tiled  
    
    return s_next_all_actions
    

In [18]:
def make_prob_actions(df_in):
    df_local = df_in.reset_index()
    prob = df_local['policy'].values
    prob = np.vstack([arr for arr in prob])
    prob = np.append(prob, [np.zeros(25)], axis=0)
    prob = prob[1:]
    return prob

In [19]:
# call predict on s_next_all_actions
# weight by prob_actions
# zero out the terminal ones
def make_exp_next_q(s_next_all_actions, prob_actions, term_indices, reg):
    if reg is None:
        return np.zeros(len(s_next_all_actions)/25)
    else:
        next_q = reg.predict(s_next_all_actions)
        next_q = next_q.reshape([-1,25])
        exp_q = np.sum(prob_actions*next_q, axis=1)
        exp_q[term_indices] = 0
        return exp_q       

In [20]:
# prepare data
# train
train_X = make_s_a(df)
train_term_indices = find_term_indices(df)
train_s_next_a_next = make_s_next_a_next(df)
train_prob_actions = make_prob_actions(df)

# val
val_X = make_s_a(val_df)
val_term_indices = find_term_indices(val_df)
val_s_next_a_next = make_s_next_a_next(val_df)
val_prob_actions = make_prob_actions(val_df)

# test
test_X = make_s_a(test_df)
test_term_indices = find_term_indices(test_df)
test_s_next_a_next = make_s_next_a_next(test_df)
test_prob_actions = make_prob_actions(test_df)

reg = None


find_term_indices, step 10000
find_term_indices, step 20000
find_term_indices, step 30000
find_term_indices, step 40000
find_term_indices, step 50000
find_term_indices, step 60000
find_term_indices, step 70000
find_term_indices, step 80000
find_term_indices, step 90000
find_term_indices, step 100000
find_term_indices, step 110000
find_term_indices, step 120000
find_term_indices, step 130000
find_term_indices, step 140000
find_term_indices, step 150000
find_term_indices, step 160000
find_term_indices, step 10000
find_term_indices, step 20000
find_term_indices, step 10000
find_term_indices, step 20000
find_term_indices, step 30000
find_term_indices, step 40000


In [21]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
train_X_quad = poly.fit_transform(train_X)
val_X_quad = poly.fit_transform(val_X)

In [22]:
train_s_next_a_next.shape

(4237375, 50)

In [ ]:
train_s_next_a_next_quad = poly.fit_transform(train_s_next_a_next)
val_s_next_a_next_quad = poly.fit_transform(val_s_next_a_next)

In [ ]:
# New training loop with quadratic features
from sklearn.linear_model import SGDRegressor
# from sklearn.svm import LinearSVR

reg = None

num_iters = 20

gamma = 0.99

for i in range(num_iters):
    next_q = make_exp_next_q(train_s_next_a_next_quad, train_prob_actions, train_term_indices, reg)
    train_y = df['reward'] + gamma * next_q
    
    # create new reg object, and then call fit on train_X and train_y
    reg = SGDRegressor(n_iter=100)
#     reg = LinearSVR(max_iter=1000)
    
    reg.fit(train_X_quad, train_y)

    val_next_q  = make_exp_next_q(val_s_next_a_next_quad, val_prob_actions, val_term_indices, reg)
    val_y = val_df['reward'] + gamma * val_next_q
    pred = reg.predict(val_X_quad)
    mean_error = np.mean((pred - val_y)**2)
    print "Mean on val set is %f, mse is %f" % (np.mean(pred),mean_error)
    

In [90]:
# New training loop
from sklearn.linear_model import SGDRegressor
# from sklearn.svm import LinearSVR

reg = None

num_iters = 20

gamma = 0.99

for i in range(num_iters):
    next_q = make_exp_next_q(train_s_next_a_next, train_prob_actions, train_term_indices, reg)
    train_y = df['reward'] + gamma * next_q
    
    # create new reg object, and then call fit on train_X and train_y
    reg = SGDRegressor(n_iter=100)
#     reg = LinearSVR(max_iter=1000)
    
    reg.fit(train_X, train_y)

    val_next_q  = make_exp_next_q(val_s_next_a_next, val_prob_actions, val_term_indices, reg)
    val_y = val_df['reward'] + gamma * val_next_q
    pred = reg.predict(val_X)
    mean_error = np.mean((pred - val_y)**2)
    print "Mean on val set is %f, mse is %f" % (np.mean(pred),mean_error)
    

Mean on val set is 1.033356, mse is 12.768726
Mean on val set is 1.829905, mse is 11.086921
Mean on val set is 2.478490, mse is 10.007288
Mean on val set is 3.274795, mse is 9.111879
Mean on val set is 4.103237, mse is 8.398450
Mean on val set is 4.873719, mse is 7.864678
Mean on val set is 5.515723, mse is 7.503094
Mean on val set is 6.115340, mse is 7.221246
Mean on val set is 6.628810, mse is 7.023555
Mean on val set is 7.116841, mse is 6.867109
Mean on val set is 7.542847, mse is 6.753096
Mean on val set is 7.945831, mse is 6.664302
Mean on val set is 8.346300, mse is 6.592087
Mean on val set is 8.601751, mse is 6.554877
Mean on val set is 8.841588, mse is 6.524314
Mean on val set is 9.023748, mse is 6.511831
Mean on val set is 9.221690, mse is 6.494081
Mean on val set is 9.448315, mse is 6.480782
Mean on val set is 9.597533, mse is 6.471679
Mean on val set is 9.491029, mse is 6.497056


In [79]:
# get final preds for val and test sets
val_q = reg.predict(val_X)
test_q = reg.predict(test_X)

np.save('val_q_learned_policy.npy', val_q)
np.save('test_q_learned_policy.npy', test_q)

In [80]:
from sklearn.externals import joblib
joblib.dump(reg, 'lr_model.pkl')

['lr_model.pkl']